In [2]:
import yfinance as yf
import pandas as pd
import concurrent.futures
from tqdm import tqdm

# Load Russell 2000 tickers from a CSV file
russell_2000_file = 'russell_2000.csv'
russell_2000_df = pd.read_csv(russell_2000_file)

# Create an empty dataframe to store the results
columns = ['Ticker', 'Company', 'Cash on Hand']
cash_df = pd.DataFrame(columns=columns)

# Function to get Cash on Hand
def get_cash_on_hand(ticker, company):
    try:
        stock = yf.Ticker(ticker)
        total_cash = stock.info.get('totalCash', None)  # Fetching total cash on hand
        return {'Ticker': ticker, 'Company': company, 'Cash on Hand': total_cash}
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")
        return {'Ticker': ticker, 'Company': company, 'Cash on Hand': None}

# List to collect data rows
data_rows = []

# Multithreading the fetching process
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(get_cash_on_hand, row['Ticker'], row['Company'])
        for index, row in russell_2000_df.iterrows()
    ]
    
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        result = future.result()
        if result['Cash on Hand'] is not None:
            data_rows.append(result)

# Create DataFrame from the collected data
cash_df = pd.DataFrame(data_rows)

# Save the DataFrame to an Excel file
output_file = 'russell_2000_cash_on_hand.xlsx'
cash_df.to_excel(output_file, index=False)

print(f"DataFrame has been saved to {output_file}")


  0%|                                          | 1/1909 [00:01<45:06,  1.42s/it]404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AAC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AAC&crumb=vrsY67BYNzn
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABMD?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABMD&crumb=vrsY67BYNzn
  8%|███▍                                    | 162/1909 [00:06<01:04, 26.92it/s]404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AVID?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AVID&crumb=vrsY67BYNzn
404 Client Error: Not Found for url: https://query2.f

DataFrame has been saved to russell_2000_cash_on_hand.xlsx
